In [5]:
import requests
from requests import Response
aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'

res: Response = requests.get(aqi_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

#自訂的欄位驗証
from pydantic import BaseModel, Field, field_validator

class Site(BaseModel):
    site_name: str = Field(alias='sitename')
    county: str
    aqi: int
    status: str
    pm25: float = Field(alias='pm2.5')
    so2: float
    co: float

    @field_validator('pm25', 'so2', 'co', mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        if value == '':
            return '0.0'
        else:
            return value

class AQI(BaseModel):
    records: list[Site]

data: AQI = AQI.model_validate_json(res.text)
print(data)

下載成功
records=[Site(site_name='屏東(枋山)', county='屏東縣', aqi=30, status='良好', pm25=2.0, so2=0.2, co=0.14), Site(site_name='大甲（日南國小）', county='臺中市', aqi=22, status='良好', pm25=7.0, so2=0.0, co=0.2), Site(site_name='新北(樹林)', county='新北市', aqi=42, status='良好', pm25=13.0, so2=0.3, co=0.21), Site(site_name='屏東（琉球）', county='屏東縣', aqi=27, status='良好', pm25=8.0, so2=0.0, co=0.06), Site(site_name='臺南（麻豆）', county='臺南市', aqi=26, status='良好', pm25=10.0, so2=0.4, co=0.11), Site(site_name='高雄（湖內）', county='高雄市', aqi=20, status='良好', pm25=4.0, so2=1.4, co=0.13), Site(site_name='彰化（員林）', county='彰化縣', aqi=36, status='良好', pm25=9.0, so2=1.1, co=0.14), Site(site_name='大城', county='彰化縣', aqi=29, status='良好', pm25=9.0, so2=0.5, co=0.03), Site(site_name='富貴角', county='新北市', aqi=30, status='良好', pm25=5.0, so2=0.0, co=0.1), Site(site_name='麥寮', county='雲林縣', aqi=21, status='良好', pm25=5.0, so2=1.0, co=0.1), Site(site_name='關山', county='臺東縣', aqi=32, status='良好', pm25=10.0, so2=0.8, co=0.1), Site(site_name='馬公', 

In [8]:
all_sites:list[dict] = data.model_dump()['records']
all_sites    #python資料結構

[{'site_name': '屏東(枋山)',
  'county': '屏東縣',
  'aqi': 30,
  'status': '良好',
  'pm25': 2.0,
  'so2': 0.2,
  'co': 0.14},
 {'site_name': '大甲（日南國小）',
  'county': '臺中市',
  'aqi': 22,
  'status': '良好',
  'pm25': 7.0,
  'so2': 0.0,
  'co': 0.2},
 {'site_name': '新北(樹林)',
  'county': '新北市',
  'aqi': 42,
  'status': '良好',
  'pm25': 13.0,
  'so2': 0.3,
  'co': 0.21},
 {'site_name': '屏東（琉球）',
  'county': '屏東縣',
  'aqi': 27,
  'status': '良好',
  'pm25': 8.0,
  'so2': 0.0,
  'co': 0.06},
 {'site_name': '臺南（麻豆）',
  'county': '臺南市',
  'aqi': 26,
  'status': '良好',
  'pm25': 10.0,
  'so2': 0.4,
  'co': 0.11},
 {'site_name': '高雄（湖內）',
  'county': '高雄市',
  'aqi': 20,
  'status': '良好',
  'pm25': 4.0,
  'so2': 1.4,
  'co': 0.13},
 {'site_name': '彰化（員林）',
  'county': '彰化縣',
  'aqi': 36,
  'status': '良好',
  'pm25': 9.0,
  'so2': 1.1,
  'co': 0.14},
 {'site_name': '大城',
  'county': '彰化縣',
  'aqi': 29,
  'status': '良好',
  'pm25': 9.0,
  'so2': 0.5,
  'co': 0.03},
 {'site_name': '富貴角',
  'county': '新北市',
  'aqi':

In [7]:
import pandas as pd
df1 = pd.DataFrame(all_sites)
df1

,site_name,county,aqi,status,pm25,so2,co
0,屏東(枋山),屏東縣,25,良好,12.0,0.7,0.27
1,大甲（日南國小）,臺中市,18,良好,7.0,0.1,0.21
2,新北(樹林),新北市,44,良好,14.0,0.4,0.30
3,屏東（琉球）,屏東縣,32,良好,9.0,0.6,0.05
4,臺南（麻豆）,臺南市,29,良好,11.0,0.7,0.20
...,...,...,...,...,...,...,...
995,頭份,苗栗縣,35,良好,11.0,3.4,0.31
996,新竹,新竹市,30,良好,7.0,1.0,0.31
997,竹東,新竹縣,36,良好,17.0,1.3,0.25
998,湖口,新竹縣,21,良好,7.0,1.0,0.13


In [10]:
df1[df1['aqi'] <= 15]

,site_name,county,aqi,status,pm25,so2,co
266,馬公,澎湖縣,15,良好,0.0,0.0,0.00
351,馬公,澎湖縣,15,良好,0.0,0.0,0.00
436,馬公,澎湖縣,15,良好,1.0,0.8,0.09
